<a href="https://colab.research.google.com/github/raz0208/Techniques-For-Text-Analysis/blob/main/FNN_SimpleImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Feedforward Neural Network (FNN)
A Feedforward Neural Network (FNN) is one of the simplest and most widely used types of artificial neural networks. It’s called "feedforward" because the data flows in one direction — from input to output — without any loops or cycles.

Here’s a quick breakdown of how it works:
- **Input Layer:** Takes in the input features (like images, text, or numerical data).
- **Hidden Layers:** One or more layers of neurons that process the input by applying weights, biases, and activation functions.
- **Output Layer:** Produces the final prediction or classification.

In an FNN, each neuron in one layer is connected to every neuron in the next layer — that’s why it’s often called a fully connected network. The network is trained using backpropagation, adjusting weights based on the error between predicted and actual outputs.

### Step 1: Import libraries and read data

In [1]:
# Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

In [2]:
# Read data (Dummy data for illustration)
data = [
    ("hello world", 0),
    ("machine learning is amazing", 1),
    ("deep learning for vision", 1),
    ("hello from the other side", 0),
]

data

[('hello world', 0),
 ('machine learning is amazing', 1),
 ('deep learning for vision', 1),
 ('hello from the other side', 0)]

### Step 2: Process data

In [7]:
# Tokenize and build vocabulary
tokens = [word for text, _ in data for word in text.split()]
vocab = {word: idx for idx, (word, _) in enumerate(Counter(tokens).items())}

print(tokens, "\n")
print(vocab, "\n")

# Indexing
def encode(text):
    return [vocab[word] for word in text.split() if word in vocab]

encoded_data = [(encode(text), label) for text, label in data]

print(encoded_data, "\n")

# Padding sequences and creating tensors
max_len = max(len(x) for x, _ in encoded_data)
X = torch.tensor([x + [0] * (max_len - len(x)) for x, _ in encoded_data])
y = torch.tensor([label for _, label in encoded_data])

X,y

['hello', 'world', 'machine', 'learning', 'is', 'amazing', 'deep', 'learning', 'for', 'vision', 'hello', 'from', 'the', 'other', 'side'] 

{'hello': 0, 'world': 1, 'machine': 2, 'learning': 3, 'is': 4, 'amazing': 5, 'deep': 6, 'for': 7, 'vision': 8, 'from': 9, 'the': 10, 'other': 11, 'side': 12} 

[([0, 1], 0), ([2, 3, 4, 5], 1), ([6, 3, 7, 8], 1), ([0, 9, 10, 11, 12], 0)] 



(tensor([[ 0,  1,  0,  0,  0],
         [ 2,  3,  4,  5,  0],
         [ 6,  3,  7,  8,  0],
         [ 0,  9, 10, 11, 12]]),
 tensor([0, 1, 1, 0]))

### Step 3: Define models
- Linear Model
- Non linear Model

In [8]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Linear FNN
class LinearFNN(nn.Module):
    def __init__(self, input_size):
        super(LinearFNN, self).__init__()
        self.fc = nn.Linear(input_size, 1)
    def forward(self, x):
        return torch.sigmoid(self.fc(x.float())).squeeze()

# Non-linear FNN
class NonLinearFNN(nn.Module):
    def __init__(self, input_size):
        super(NonLinearFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 1)
    def forward(self, x):
        x = self.relu(self.fc1(x.float()))
        return torch.sigmoid(self.fc2(x)).squeeze()

### Step 4: Define training and testing
- Training and testing linear model
- Traninf and testing nonlinear model

In [12]:
# Class and function to train FNN
class Trainer:
    def __init__(self, model, lr=0.001):
        self.model = model
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(model.parameters(), lr=lr)

    def train(self, X, y, epochs=10):
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            outputs = self.model(X)
            loss = self.criterion(outputs, y.float())
            loss.backward()
            self.optimizer.step()
            print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

    def evaluate(self, X, y):
        with torch.no_grad():
            outputs = self.model(X)
            predictions = (outputs > 0.5).int()
            predictions = predictions.view(-1).cpu().numpy()  # Ensure 1D array
            y = y.view(-1).cpu().numpy()  # Ensure 1D array
            acc = accuracy_score(y, predictions)
            print(f"Accuracy: {acc:.4f}")

### Step 5: Train and test models

In [13]:
# Train and test the model
input_size = max_len

print("Training Linear Model:")
linear_model = LinearFNN(input_size)
trainer = Trainer(linear_model)
trainer.train(X_train, y_train)
trainer.evaluate(X_test, y_test)

Training Linear Model:
Epoch 1, Loss: 1.9917
Epoch 2, Loss: 1.9835
Epoch 3, Loss: 1.9752
Epoch 4, Loss: 1.9670
Epoch 5, Loss: 1.9588
Epoch 6, Loss: 1.9506
Epoch 7, Loss: 1.9425
Epoch 8, Loss: 1.9343
Epoch 9, Loss: 1.9261
Epoch 10, Loss: 1.9180
Accuracy: 1.0000


In [14]:
print("\nTraining Non-Linear Model:")
non_linear_model = NonLinearFNN(input_size)
trainer = Trainer(non_linear_model)
trainer.train(X_train, y_train)
trainer.evaluate(X_test, y_test)


Training Non-Linear Model:
Epoch 1, Loss: 1.3077
Epoch 2, Loss: 1.2796
Epoch 3, Loss: 1.2518
Epoch 4, Loss: 1.2245
Epoch 5, Loss: 1.1978
Epoch 6, Loss: 1.1715
Epoch 7, Loss: 1.1456
Epoch 8, Loss: 1.1202
Epoch 9, Loss: 1.0954
Epoch 10, Loss: 1.0712
Accuracy: 1.0000
